In [1]:
# -*- coding: utf-8 -*-
import random
import jpype as jp
import pandas as pd
from pandas import DataFrame
import import_ipynb
import Module1

from os.path import join
from typing import List

from jpype import (JClass, JString, getDefaultJVMPath, java, shutdownJVM,
                   startJVM)


## Zemberek: Finding POS Tag Example
# Documentation: https://github.com/ahmetaa/zemberek-nlp/tree/master/morphology
# Java Code Example: https://github.com/ahmetaa/zemberek-nlp/blob/master/examples/src/main/java/zemberek/examples/morphology/FindPOS.java

# Relative path to Zemberek .jar
ZEMBEREK_PATH = '/home/busra/zemberek-full.jar'#download zemberek jar and give it's path name

# Start the JVM
jp.startJVM(jp.getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (ZEMBEREK_PATH))

# Import the required Java classes
TurkishMorphology = jp.JClass('zemberek.morphology.TurkishMorphology')

# Instantiating the morphology class with the default RootLexicon
morphology = TurkishMorphology.createWithDefaults()
stemList = []
lineList = []
# A dummy sentence to work on

# A list to store primary POS tags
pos_Noun_Prop = []#noun
pos_Verb = []#verbs
pos_Adj = []#adjective
pos_Adv = []#adverb
pos_ben = []#ben
pos_sen = []#sen
pos_o = []#o
pos_biz = []#biz
pos_siz = []#siz
pos_onlar = []#onlar
pos_object = []#object
pos_time = []#bugün yarın gün akşam
pos_demons = []# ondan onu bunu şunu ona buna şuna şundan bundan
pos_Conj = []#fakat ama eğer


positive_negatives: List[JString] = [JString(''), JString('Neg')]
times: List[JString] = [
    'Imp', 'Aor', 'Past', 'Prog1', 'Prog2', 'Narr', 'Fut'#time suffixes
]
people: List[JString] = [
    'A1sg', 'A2sg', 'A3sg', 'A1pl', 'A2pl', 'A3pl'#person suffixes
]


def analize(analysis, word):#analize words
    for i, a in enumerate(analysis):
        #print('Analysis %d: %s' % (i+1, a))#a kullanılcak
        #print('Primary POS %d: %s' % (i+1, a.getPos()))
        if 'Noun' in str(a):
            if 'A1sg' in str(a):
                pos_ben.append(word)#words about ben
            if 'A2sg' in str(a):
                pos_sen.append(word)#words about sen
            if 'A3sg' in str(a) or 'Det' in str(a):#words about o
                pos_o.append(word)
            if 'A1pl' in str(a) and 'Pron' in str(a) :#words about biz
                pos_biz.append(word)
            if 'A2pl' in str(a):#words about siz
                pos_siz.append(word)
            if 'A3pl' in str(a) and 'Pron' in str(a):#words about onlar
                pos_onlar.append(word)
            if 'Noun,Prop' in str(a):#words about other nouns
                pos_Noun_Prop.append(word)
            if 'Noun,Time' in str(a):#time words
                pos_time.append(word)
            if 'Pron,Demons' in str(a):#words about onu,bunu,şunu,ondan,bundan,şundan etc.
                pos_Demons.append(word)
            else:
                pos_object.append(word)            

        elif 'Verb' in str(a):#verbs
            word_temp = str(a)[1:]
            word_temp = word_temp.split(":")
            word = word_temp[0]
            if word[-3:] == 'mak' or word[-3:] == 'mek':#verbs that are stem
                pos_Verb.append(word)
               
        elif 'Adj' in str(a):#adjective
            pos_Adj.append(word)
        elif 'Adv' in str(a):#adverb
            pos_Adv.append(word)
        elif 'Conj'in str(a):#conjuction
            pos_Conj.append(word)
            
    
def get_list(dict):
    return dict.keys()


def classify_word():#take words one by one from corpus and put them to word_list.Then analysis all words as morphological
    all_files_path = Module1.get_list_of_files("/home/busra/1150haber")# corpus that have datas
    Module1.create_word_list(all_files_path)
    word_list = Module1.word_list.keys()
    for word in word_list:
        analysis = morphology.analyzeAndDisambiguate(word).bestAnalysis()
        analize(analysis,word)

if __name__ == '__main__':
    print("wait for a one minute for running ...")
    classify_word()
    i = 0
    while(i<10):
        verb = random.choice(pos_Verb)#choose random 10 verbs from Verb list



        morphology: TurkishMorphology = (
            TurkishMorphology.builder().setLexicon(verb).disableCache().build()
        )

        stem: str = verb

        for pos_neg in positive_negatives:#add positive and negative suffiex to chosen words
            for time in times:#add time suffixes to chosen verbs
                for person in people:#add person suffixes to chosen verbs
                    seq: java.util.ArrayList = java.util.ArrayList()#put this all suffixes to seq
                    if pos_neg:
                        seq.add(JString(pos_neg))
                    if time:
                        seq.add(JString(time))
                    if person == 'A2sg' or person == 'A1sg' or person == 'A3sg'  or person == 'A1pl' or person == 'A2pl' or person == 'A3pl':

                        seq.add(JString(person))

                    results = list(morphology.getWordGenerator().generate(#generate new conjugate words from verb stems with suffixes in seq
                        JString(stem),
                        seq
                    ))
                    if not results:

                        #print((
                           # f'Cannot generate Stem = ["{stem}"]'
                            #f'\n | Morphemes = {[str(morph) for morph in seq]}'
                        #))
                        continue
                    for morph in seq:#for all times and person verb conjugations
                        if 'A1sg' in str(morph):#new sentences for A1sg person grammatical order according to Turkish language
                            for result in results:
                                print('Ben'+ " " + str(result.surface))
                                print('Ben'+ " " + random.choice(pos_ben) + " " + str(result.surface))
                                print('Ben'+ " " + random.choice(pos_ben) + " "+random.choice(pos_time)+" "+ str(result.surface))
                                print('Ben'+ " " + random.choice(pos_ben) + " " +random.choice(pos_Adv)+" "+ str(result.surface))
                                print('Ben'+ " " + random.choice(pos_ben) + " " +random.choice(pos_Adv)+" "+ str(result.surface) +" "+random.choice(pos_Conj)+ " " + random.choice(pos_ben) + " " +random.choice(pos_Adv)+" "+ str(result.surface))
                                print('Ben'+ " " + random.choice(pos_ben) + " "+random.choice(pos_Adj)+" "+random.choice(pos_object)+ " " +random.choice(pos_Adv)+" "+ str(result.surface))

                        elif 'A2sg' in str(morph):#new sentences for A2sg person grammatical order according to Turkish language
                            for result in results:
                                print('Sen'+ " " + str(result.surface))
                                print('Sen'+ " " +random.choice(pos_sen) + " " + str(result.surface))
                                print('Sen'+ " " + random.choice(pos_sen) + " "+random.choice(pos_time)+" "+ str(result.surface))
                                print('Sen'+ " " + random.choice(pos_sen) + " " +random.choice(pos_Adv)+" "+ str(result.surface))
                                print('Sen'+ " " + random.choice(pos_sen) + " " +random.choice(pos_Adv)+" "+ str(result.surface) +" "+random.choice(pos_Conj)+ " " + random.choice(pos_sen) + " " +random.choice(pos_Adv)+" "+ str(result.surface))
                                print('Sen'+ " " + random.choice(pos_sen) + " "+random.choice(pos_Adj)+" "+random.choice(pos_object)+ " " +random.choice(pos_Adv)+" "+ str(result.surface))
                        elif 'A3sg' in str(morph):#new sentences for A3sg person grammatical order according to Turkish language
                            for result in results:
                                print('O'+ " " + str(result.surface))
                                print('O'+ " " + random.choice(pos_o) + " " + str(result.surface))
                                print('O'+ " " + random.choice(pos_o) + " "+random.choice(pos_time)+" "+ str(result.surface))
                                print('O'+ " " + random.choice(pos_o) + " " +random.choice(pos_Adv)+" "+ str(result.surface))
                                print('O'+ " " + random.choice(pos_o) + " " +random.choice(pos_Adv)+" "+ str(result.surface) +" "+random.choice(pos_Conj)+ " " + random.choice(pos_o) + " " +random.choice(pos_Adv)+" "+ str(result.surface))
                                print('O'+ " " + random.choice(pos_o) + " "+random.choice(pos_Adj)+" "+random.choice(pos_object)+ " " +random.choice(pos_Adv)+" "+ str(result.surface))
                        elif 'A1pl' in str(morph):#new sentences for A1pl person grammatical order according to Turkish language
                            for result in results:
                                print('Biz'+ " " + str(result.surface))
                                print('Biz'+ " " +random.choice(pos_biz) + " " + str(result.surface))
                                print('Biz'+ " " + random.choice(pos_biz) + " "+random.choice(pos_time)+" "+ str(result.surface))
                                print('Biz'+ " " + random.choice(pos_biz) + " " +random.choice(pos_Adv)+" "+ str(result.surface))
                                print('Biz'+ " " + random.choice(pos_biz) + " " +random.choice(pos_Adv)+" "+ str(result.surface) +" "+random.choice(pos_Conj)+ " " + random.choice(pos_biz) + " " +random.choice(pos_Adv)+" "+ str(result.surface))
                                print('Biz'+ " " + random.choice(pos_biz) + " "+random.choice(pos_Adj)+" "+random.choice(pos_object)+ " " +random.choice(pos_Adv)+" "+ str(result.surface))
                        elif 'A2pl' in str(morph):#new sentences for A2pl person grammatical order according to Turkish language
                            for result in results:
                                print('Siz'+ " " + str(result.surface))
                                print('Siz'+ " " +random.choice(pos_siz) + " " + str(result.surface))
                                print('Siz'+ " " + random.choice(pos_siz) + " "+random.choice(pos_time)+" "+ str(result.surface))
                                print('Siz'+ " " + random.choice(pos_siz) + " " +random.choice(pos_Adv)+" "+ str(result.surface))
                                print('Siz'+ " " + random.choice(pos_siz) + " " +random.choice(pos_Adv)+" "+ str(result.surface) +" "+random.choice(pos_Conj)+ " " + random.choice(pos_siz) + " " +random.choice(pos_Adv)+" "+ str(result.surface))
                                print('Siz'+ " " + random.choice(pos_siz) + " "+random.choice(pos_Adj)+" "+random.choice(pos_object)+ " " +random.choice(pos_Adv)+" "+ str(result.surface))
                        elif 'A3pl' in str(morph):#new sentences for A3pl person grammatical order according to Turkish language
                            for result in results:
                                print('Onlar'+ " " + str(result.surface))
                                print('Onlar'+ " " +random.choice(pos_onlar) + " " + str(result.surface))
                                print('Onlar'+ " " + random.choice(pos_onlar) + " "+random.choice(pos_time)+" "+ str(result.surface))
                                print('Onlar'+ " " + random.choice(pos_onlar) + " " +random.choice(pos_Adv)+" "+ str(result.surface))
                                print('Onlar'+ " " + random.choice(pos_onlar) + " " +random.choice(pos_Adv)+" "+ str(result.surface) +" "+random.choice(pos_Conj)+ " " + random.choice(pos_onlar) + " " +random.choice(pos_Adv)+" "+ str(result.surface))
                                print('Onlar'+ " " + random.choice(pos_onlar) + " "+random.choice(pos_Adj)+" "+random.choice(pos_object)+ " " +random.choice(pos_Adv)+" "+ str(result.surface))

        i=i+1 #do it for ten conjugate verbs randomly chosen from pos_Verb list
# Shutting down the JVM
jp.shutdownJVM()

importing Jupyter notebook from Module1.ipynb


/home/busra/PycharmProjects/Piton/myenv/lib/python3.6/site-packages/jpype/_core.py:218: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


wait for a one minute for running ...
Sen tartış
Sen yakınlaşma tartış
Sen örgütlenme YILA tartış
Sen Aldatılan resmen tartış
Sen getirilen yıllarca tartış Veya Göksen yine tartış
Sen aldatılan ayrı kapasitesine birden tartış
Sen tartışsana
Sen sıkılaşma tartışsana
Sen gençleştirme dönemiydi tartışsana
Sen Lütfusun Mutlaka tartışsana
Sen şartlan Hâlâ tartışsana Ancak kömürleşme fiilen tartışsana
Sen Göksen köhne başvuruda pek tartışsana
O tartışsın
O asansör tartışsın
O istihbaratı önceye tartışsın
O macerasının hele tartışsın
O yeteneğine Belki tartışsın ancak kaçakçılığın Halen tartışsın
O Day saçık sınıfı Şimdilik tartışsın
Siz tartışın
Siz Meraklanmayın tartışın
Siz mankeniydiniz Akşamları tartışın
Siz içindesiniz Erken tartışın
Siz hocasınız Hayli tartışın ANCAK Erkekseniz kulaktan tartışın
Siz yerdesiniz sağlam ölmesi Gerçekten tartışın
Siz tartışınız
Siz torunusunuz tartışınız
Siz başındaysanız sonraki tartışınız
Siz başındaysanız Acaba tartışınız
Siz yararlanın boyca tartışınız

Ben beğenmektim
Ben zorundayım beğenmektim
Ben Umutluyum günün beğenmektim
Ben sinirlendim Zaten beğenmektim
Ben camiadayım cidden beğenmektim HEM kızıydım aracılığıyla beğenmektim
Ben görüşündeyim sulu savunduklarını müştereken beğenmektim
Sen beğenmektin
Sen çıkartılan beğenmektin
Sen bilinçlendirme Haftanın beğenmektin
Sen kapılan boyca beğenmektin
Sen örgütlenme nasılsa beğenmektin hatta gerçekleşme YERİNE beğenmektin
Sen gençleştirme SANAL Bülteni böylelikle beğenmektin
O beğenmekti
O pozisyonun beğenmekti
O taksitte haftalardır beğenmekti
O Modacı sıkça beğenmekti
O SAADET Kanunen beğenmekti AMA başladığının Eskiden beğenmekti
O Pencere Derin Önergelerin Yine beğenmekti
Biz beğenmektik
Biz Bizimkiler beğenmektik
Biz bizimkiler yılı beğenmektik
Biz Bizimkiler asla beğenmektik
Biz Bizimkiler sonradan beğenmektik yani bizimkinden bizzat beğenmektik
Biz bizimkiler durgun kademeli direkt beğenmektik
Siz beğenmektiniz
Siz şirketteydiniz beğenmektiniz
Siz yoldasınız YILDA beğenmektiniz


Ben etmektim
Ben yaşındaydım etmektim
Ben kürdüm Öğlene etmektim
Ben kanaatindeyim Aniden etmektim
Ben STARDIM başbaşa etmektim ancak kızıyım birgün etmektim
Ben disiplinliydim muhammen turnuvasında hani etmektim
Sen etmektin
Sen belirtilen etmektin
Sen arasındasın önceden etmektin
Sen kapılan yalnızca etmektin
Sen normalleşme Bazen etmektin HEM mıknatıslanma Ayrıca etmektin
Sen kömürleşme Müthiş Esther giderayak etmektin
O etmekti
O çıkar etmekti
O sınırlaması an etmekti
O sezonundaki Sadece etmekti
O gezilmesi Hele etmekti Lakin Mevsim EN etmekti
O dikkate otomatik kişiler başbaşa etmekti
Biz etmektik
Biz Bizimkiler etmektik
Biz bizimkinden yıla etmektik
Biz bizimkiler Şahsen etmektik
Biz Bizimkiler Evet etmektik Da Bizimkiler boyca etmektik
Biz bizimkinden Demokrat güney herhalde etmektik
Siz etmektiniz
Siz Erkekseniz etmektiniz
Siz şirketteydiniz Geceleri etmektiniz
Siz güzelleştirin daha etmektiniz
Siz Meraklanmayın gittikçe etmektiniz Hem zorundasınız peşinen etmektiniz
Siz yarar

Ben getirmektim
Ben Umutluyum getirmektim
Ben insandım haftada getirmektim
Ben mankendim Geri getirmektim
Ben ucundayım Belki getirmektim şayet görüşündeyim Yine getirmektim
Ben kanaatindeyim nükleer Pachulia Böylece getirmektim
Sen getirmektin
Sen getirilen getirmektin
Sen belirtilen yaz getirmektin
Sen yargılan sadece getirmektin
Sen sarsılan galiba getirmektin DE bilgilendirme ÇABUK getirmektin
Sen pıhtılaşma Birleşik Disiplin aniden getirmektin
O getirmekti
O gübre getirmekti
O Meditasyonda yıllarında getirmekti
O siyasetçiler külliyen getirmekti
O havuzlarında aslında getirmekti şayet beldedeki YERİNE getirmekti
O taş Külkedisi ilköğretim erken getirmekti
Biz getirmektik
Biz Bizimkiler getirmektik
Biz Bizimkiler Sabahları getirmektik
Biz bizimkinden demek getirmektik
Biz bizimkinden Artık getirmektik ila bizimkiler Geri getirmektik
Biz Bizimkiler Rasyonel Lebedev Aniden getirmektik
Siz getirmektiniz
Siz değerlendirebilirsiniz getirmektiniz
Siz mankeniydiniz döneme getirmektiniz
Si

Sen harcamazsın
Sen Göksen harcamazsın
Sen fiyatlandırma Bugünden harcamazsın
Sen yakılan geçenlerde harcamazsın
Sen ışıklandırma Kısacası harcamazsın ki kapılan müştereken harcamazsın
Sen şekillendirme beyaz askerine gittikçe harcamazsın
O harcamaz
O çözümünü harcamaz
O suskunluğunu GÜN harcamaz
O gözlemlenen Galiba harcamaz
O Mersin Genelde harcamaz HEM gelişimcisi bizzat harcamaz
O sigara kırmızı konserlerime Hemen harcamaz
Biz harcamayız
Biz Bizimkiler harcamayız
Biz Bizimkiler sonrasına harcamayız
Biz bizimkinden Sadece harcamayız
Biz Bizimkiler asaleten harcamayız Çünkü bizimkiler adeta harcamayız
Biz Bizimkiler kısa matbaacılığa Genellikle harcamayız
Siz harcamazsınız
Siz hocasınız harcamazsınız
Siz mankeniydiniz yıllardır harcamazsınız
Siz taksitlendirebilirsiniz Keza harcamazsınız
Siz zorundasınız Nitekim harcamazsınız ki Meraklanmayın kendiliğinden harcamazsınız
Siz yerdesiniz insani müdürlüğüne tahminen harcamazsınız
Onlar harcamazlar
Onlar partilerinki harcamazlar
Onlar biz